# Tables - Storing and reading tables with Pandas

In [23]:
import quasardb
import quasardb.pandas as qdbpd
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
sample_size=10
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [24]:
print(quasardb.version())
c = quasardb.Cluster("qdb://qdb-server:2836")


3.5.0master


In [25]:
bidask=np.array([['2019-03-13 09:30:00',10,11],
               ['2019-03-13 09:30:01',11,12],
              ['2019-03-13 09:30:02',10,11]])

trade=np.array([['2019-03-13 09:30:01',12]])




In [26]:
df_bidask = pd.DataFrame( data=bidask[:,1:].astype('int64'),
            index=bidask[0:,0].astype('datetime64[ns]'),
            columns=['bid','ask'])
df_trade = pd.DataFrame( data=trade[:,1:].astype('int64'),
            index=trade[0:,0].astype('datetime64[ns]'),
            columns=['trade'])

In [27]:
df_bidask

,bid,ask
2019-03-13 09:30:00,10,11
2019-03-13 09:30:01,11,12
2019-03-13 09:30:02,10,11


In [28]:
qdbpd.write_dataframe(df_bidask, c, "bidask",create=True)
qdbpd.write_dataframe(df_trade, c, "trades",create=True)

In [29]:
query= qdbpd.query(c,"SELECT * FROM bidask RIGHT ASOF JOIN trades")
query

{'trades':                      bid  ask  trade
 2019-03-13 09:30:01  0.0  0.0     12,
 'bidask':                      bid  ask  trade
 2019-03-13 09:30:01   11   12    0.0}

In [ ]:
pd.mer

In [30]:
query_classic= c.query("SELECT * FROM bidask RIGHT ASOF JOIN trades")
query_result = query_classic.run()

In [31]:
tmp = dict()
for col in query_result.tables['ts_pandas_1']:
    tmp[col.name] = col.data
result_as_pandas = pd.DataFrame.from_dict(tmp, orient='columns').set_index('$timestamp')
assert result_as_pandas['close'].size == sample_size
result_as_pandas[:5]

KeyError: 'ts_pandas_1'

In [32]:
query_result.tables

{'trades': [<quasardb.quasardb.Query.ColumnResult at 0x7fd444f8ea08>,
 'bidask': [<quasardb.quasardb.Query.ColumnResult at 0x7fd444f8e688>,
  <quasardb.quasardb.Query.ColumnResult at 0x7fd444f8e998>]}

In [17]:
#tearing down data for this sample
table = c.ts("bidask")
table.remove()
table = c.ts("trades")
table.remove()